<a href="https://colab.research.google.com/github/Anubhav3084/Deep-Learning/blob/main/Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import keras
from keras.datasets import cifar10

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [11]:
type(x_train)

numpy.ndarray

In [12]:
type(y_train)

numpy.ndarray

In [13]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [14]:
cifar10_class_names = {
    0: "Plane",
    1: "Car",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Boat",
    9: "Truck"
}

* Normalize data set to 0-to-1 range

In [15]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

x_train = x_train / 255
x_test = x_test / 255

* Convert class vectors to binary class matrix

In [18]:
from tensorflow.keras.utils import to_categorical

In [19]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [24]:
y_train.shape

(50000, 10)

## **Create Model**

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [35]:
model = Sequential()

In [36]:
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu", input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation="relu"))

model.add(MaxPool2D(2, 2))

model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))

model.add(MaxPool2D(2, 2))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(10, activation="softmax"))

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_10 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 15, 15, 64)        18496     
                                                                 
 conv2d_12 (Conv2D)          (None, 13, 13, 64)        36928     
                                                      

## **Compile the model**

In [41]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
    )

## **Training and Saving the model**

In [42]:
model.fit(
    x_train, 
    y_train, 
    batch_size=64, 
    epochs=30,  # for large dataset we use low epochs
    validation_data=(x_test, y_test),
    shuffle=True
    )

Epoch 1/30
782/782 [==============================] - 18s 9ms/step - loss: 1.5712 - accuracy: 0.4227 - val_loss: 1.2265 - val_accuracy: 0.5533
Epoch 2/30
782/782 [==============================] - 6s 8ms/step - loss: 1.1765 - accuracy: 0.5820 - val_loss: 1.0428 - val_accuracy: 0.6335
Epoch 3/30
782/782 [==============================] - 6s 8ms/step - loss: 1.0015 - accuracy: 0.6467 - val_loss: 0.8704 - val_accuracy: 0.6961
Epoch 4/30
782/782 [==============================] - 6s 8ms/step - loss: 0.8871 - accuracy: 0.6879 - val_loss: 0.8005 - val_accuracy: 0.7219
Epoch 5/30
782/782 [==============================] - 6s 8ms/step - loss: 0.8228 - accuracy: 0.7139 - val_loss: 0.7482 - val_accuracy: 0.7373
Epoch 6/30
782/782 [==============================] - 6s 8ms/step - loss: 0.7671 - accuracy: 0.7316 - val_loss: 0.7062 - val_accuracy: 0.7564
Epoch 7/30
782/782 [==============================] - 6s 8ms/step - loss: 0.7206 - accuracy: 0.7467 - val_loss: 0.7052 - val_accuracy: 0.7590
Epoch

In [43]:
from pathlib import Path

In [45]:
# save neural network structure

model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# save trained weights

model.save_weights("model_weights.h5")

## **Predicting using trained model**

In [46]:
f = Path("model_structure.json")
model_structure_2 = f.read_text()

In [48]:
from keras.models import model_from_json

In [49]:
model_2 = model_from_json(model_structure_2)

In [50]:
model_2.load_weights("model_weights.h5")

In [52]:
from keras.preprocessing import image
import numpy as np

In [55]:
img = image.load_img("cat.png", target_size=(32, 32)) # load image with the target size as 32 x 32 pixels

In [56]:
image_to_test = image.img_to_array(img) / 255   # convert to numpy array and normalize it to 0-to-1

In [57]:
# add a dimension for the batch size (batch_size, dimensions_of_image)
list_of_images = np.expand_dims(image_to_test, axis=0)

In [58]:
results = model.predict(list_of_images)

In [59]:
results

array([[2.4843871e-08, 1.7341864e-12, 7.7713412e-05, 9.9708956e-01,
        1.5435751e-03, 1.2376400e-03, 5.6273939e-06, 4.5913090e-05,
        3.5463691e-11, 1.5629902e-09]], dtype=float32)

In [62]:
single_result = results[0]
single_result

array([2.4843871e-08, 1.7341864e-12, 7.7713412e-05, 9.9708956e-01,
       1.5435751e-03, 1.2376400e-03, 5.6273939e-06, 4.5913090e-05,
       3.5463691e-11, 1.5629902e-09], dtype=float32)

In [63]:
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

In [65]:
class_label = cifar10_class_names[most_likely_class_index]
print("This image is predicted as : {} with a likelihood of : {:2f}".format(class_label, class_likelihood))

This image is predicted as : Cat with a likelihood of : 0.997090


## **Fine-Tuning Pre-Trained Neural Netowrks**
* VGG (University of Oxford)
* ResNet-50 (Microsoft Research)
* Inception v3 (Google)
* MobileNet (Google)
* NASNet (Google)
### **Two Uses**
* Use a trained model directly to do image recognition
* **Transfer Learning :** Adapt an existing model to recognize new types of objects instead of starting from scratch

In [66]:
from keras.applications import vgg16

In [67]:
model = vgg16.VGG16()

553476096/553467096 [==============================] - 6s 0us/step


In [68]:
img = image.load_img("bay.jpg", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = vgg16.preprocess_input(x)   # builtin normalizer in VGG

In [69]:
predictions = model.predict(x)

In [70]:
predicted_classes = vgg16.decode_predictions(predictions, top=9)

49152/35363 [=========================================] - 0s 0us/step


In [71]:
print("Top predictions in the image are : ")

for imagenet_id, name, likelihood in predicted_classes[0]:
    print("Prediction: {} - {:2f}".format(name, likelihood))

Top predictions in the image are : 
Prediction: seashore - 0.395212
Prediction: promontory - 0.326130
Prediction: lakeside - 0.119613
Prediction: breakwater - 0.062801
Prediction: sandbar - 0.045267
Prediction: cliff - 0.011845
Prediction: dock - 0.009196
Prediction: boathouse - 0.003278
Prediction: valley - 0.003194


### **Transfer Learning**

In [72]:
dog_path = Path("training_folder") / "dogs"
not_dog_path = Path("training_folder") / "not_dogs"

In [73]:
images = []
labels = []

In [76]:
for img in not_dog_path.glob("*.png"):
    # load the image
    img = image.load_img(img)
    # convert image to numpy array
    image_array = image.img_to_array(img)
    # add the image to the list
    images.append(image_array)
    labels.append(0)

for img in dog_path.glob("*.png"):
    # load the image
    img = image.load_img(img)
    # convert image to numpy array
    image_array = image.img_to_array(img)
    # add the image to the list
    images.append(image_array)
    labels.append(1)

UnidentifiedImageError: ignored